In [3]:
# ! pip install langgraph

In [2]:
import os
os.environ["OPENAI_API_KEY"] = key  # reemplaza con tu clave real si es necesario




## Usando LangGraph para Agregar Memoria a tu Chatbot

En el Capítulo 3, aprendiste cómo proporcionar a tu aplicación de chatbot con IA un contexto actualizado y relevante. Esto le permite generar respuestas precisas basadas en lo que el usuario escribe. Pero eso no es suficiente para tener una aplicación lista para producción. ¿Cómo puedes hacer que tu chatbot realmente converse de ida y vuelta con el usuario, recordando lo que se ha dicho antes?

Los modelos de lenguaje grandes (**LLMs**) son **sin estado**, lo que significa que cada vez que se les da un nuevo mensaje, no recuerdan el mensaje anterior ni sus propias respuestas. Para que puedan “recordar” conversaciones pasadas, necesitamos un sistema de memoria que lleve el seguimiento de los intercambios anteriores y del contexto relevante. Esta información histórica puede incluirse en el prompt final que se envía al modelo, dándole así una “memoria”.

---

## Construyendo un Sistema de Memoria para un Chatbot

Hay dos decisiones clave que tomar al diseñar un sistema de memoria:

1. **Cómo se almacena el estado**
2. **Cómo se consulta ese estado**

Una forma simple y efectiva de construir un sistema de memoria es guardar todo el historial de la conversación entre el usuario y el modelo, y reutilizarlo. El estado de este sistema puede:

- Guardarse como una lista de mensajes (ver Capítulo 1 para más detalles sobre el formato)
- Actualizarse agregando los mensajes más recientes después de cada turno
- Insertarse en el prompt para que el modelo lo tenga en cuenta

---

## Ejemplo de implementación simple con LangChain



In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# Definimos el prompt que incluye un mensaje del sistema y un marcador para los mensajes anteriores
prompt = ChatPromptTemplate.from_messages([
    ("system", """Eres un asistente útil. Responde todas las preguntas lo mejor que puedas."""),
    ("placeholder", "{messages}"),
])

# Instanciamos el modelo de chat
model = ChatOpenAI()

# Encadenamos el prompt y el modelo
chain = prompt | model

# Simulamos una conversación previa
respuesta = chain.invoke({
    "messages": [
        ("human", "Traduce esta oración del inglés al francés: I love programming."),
        ("ai", "J'adore programmer."),
        ("human", "¿Qué acabas de decir?"),
    ],
})

# Mostramos la respuesta generada por el modelo
print(respuesta.content)

Acabo de decir en francés "J'adore programmer", que significa "Me encanta programar" en español.




## Resultado esperado:

```
Dije: "J'adore programmer", que significa "I love programming" en francés.
```

---

## Limitaciones y desafíos en producción

Este enfoque simple funciona, pero en producción enfrentarás algunos retos adicionales:

- Es necesario actualizar la memoria **después de cada interacción**, de forma **atómica** (registrar tanto la pregunta como la respuesta).
- Se recomienda almacenar la memoria en un sistema duradero, como una base de datos relacional.
- Se debe controlar **cuántos mensajes** se almacenan y cuántos se usan en cada nueva interacción.
- Es útil poder **inspeccionar y modificar el estado** (la lista de mensajes) desde fuera del flujo de ejecución del modelo.





## Introducción a LangGraph

A partir de este capítulo y durante los siguientes, comenzaremos a utilizar **LangGraph**, una biblioteca de código abierto creada por **LangChain**. LangGraph está diseñada para permitir a los desarrolladores implementar **arquitecturas cognitivas con múltiples actores, múltiples pasos y con estado**, conocidas como *graphs* o **grafos**.

Eso puede sonar como muchos conceptos en una sola oración, así que vamos a desglosarlo.

---

### Aplicaciones con múltiples actores

![Figura 4-3: De aplicaciones con un solo actor a aplicaciones con múltiples actores]

Al igual que un equipo de especialistas puede lograr cosas que una sola persona no podría, las aplicaciones basadas en LLMs también pueden beneficiarse cuando se combinan con otras herramientas. Por ejemplo:

- Un **prompt de LLM** es excelente para generación de texto o planeación de tareas.
- Un **buscador web** es ideal para encontrar hechos actuales.

Cuando combinas ambos —e incluso varios LLMs diferentes entre sí— puedes crear aplicaciones innovadoras como **Perplexity** o **Arc Search**, que combinan estos elementos para lograr algo mucho más potente.

Así como los equipos humanos necesitan **coordinación**, una aplicación con múltiples actores también necesita una **capa de coordinación** para:

- Definir los actores involucrados (los **nodos** del grafo) y cómo se comunican entre sí (las **aristas**).
- Programar la ejecución de cada actor en el momento adecuado, incluso en **paralelo** y de forma **determinista**.

---

### Aplicaciones con múltiples pasos

![Figura 4-4: De múltiples actores a múltiples pasos]

Cuando un actor entrega trabajo a otro (por ejemplo, un LLM que hace una pregunta a una herramienta de búsqueda), el sistema necesita saber:

- En qué orden ocurren los eventos.
- Cuántas veces se llama a cada actor.
- Cuándo se detiene el proceso.

Esto se puede modelar como una **secuencia de pasos discretos**. Cada paso representa una transferencia de trabajo, hasta que ya no quedan tareas pendientes y se llega a un resultado final.

---

### Aplicaciones con estado

![Figura 4-5: De múltiples pasos a aplicaciones con estado]

Para coordinar acciones entre pasos, se necesita **mantener un estado**. Si no se hace esto, el LLM daría la misma respuesta cada vez que se le llama. Al centralizar el estado:

- Se puede **almacenar un snapshot** del estado en cualquier momento.
- Se puede **pausar y reanudar** la ejecución, útil para recuperación de errores.
- Se pueden **integrar humanos en el flujo de trabajo** (más sobre esto en el Capítulo 8).

---

## Componentes de un grafo en LangGraph

Cada grafo en LangGraph tiene tres elementos clave:

1. **Estado (State)**  
   Datos que entran a la aplicación y se modifican mientras el grafo está en ejecución.

2. **Nodos (Nodes)**  
   Pasos individuales representados como funciones Python. Cada función recibe el estado actual, lo modifica (si es necesario) y devuelve una nueva versión del estado.

3. **Aristas (Edges)**  
   Conexiones entre los nodos. Estas pueden ser:
   - **Fijas**: por ejemplo, después del nodo B siempre va el nodo D.
   - **Condicionales**: se evalúa una función para decidir a qué nodo ir después.

LangGraph también permite visualizar estos grafos y ofrece herramientas útiles para depuración y despliegue en producción a gran escala.

---

## Instalación de LangGraph

Si seguiste las instrucciones del Capítulo 1, ya deberías tener LangGraph instalado. Si no, puedes instalarlo con:

```bash
pip install langgraph
```

---

## Primer ejemplo: Un chatbot simple con LangGraph

Para familiarizarnos con LangGraph, construiremos un chatbot simple. Este responderá directamente a los mensajes del usuario utilizando un solo LLM. Aunque es un ejemplo sencillo, ilustra los **principios clave** de LangGraph:

- Cómo se define un nodo.
- Cómo se actualiza el estado.
- Cómo fluye la información en el grafo.


Claro, Omar. Aquí tienes el contenido traducido, reorganizado y con **todo el código Python en una sola celda ejecutable** al final, como lo pediste:

---

## 🧠 Creación de un `StateGraph` con LangGraph (Python)

### 📋 Descripción paso a paso

Para comenzar a usar **LangGraph**, lo primero que debes hacer es definir el **estado del grafo**. Este estado define:

1. La estructura del estado compartido entre nodos (por ejemplo, un historial de mensajes).
2. Cómo se actualiza cada parte del estado cuando un nodo devuelve un nuevo valor.
3. En este ejemplo, usamos `add_messages`, que indica que los mensajes nuevos deben **agregarse** a la lista existente, en lugar de sobrescribirla.

### 💬 Estado y Nodo

Definimos una clase `State` que representa el estado compartido y un nodo llamado `chatbot`, que usa un modelo de lenguaje (`ChatOpenAI`) para generar una respuesta basada en los mensajes anteriores. El resultado se agrega nuevamente al estado gracias a la anotación `add_messages`.

### 🧩 Conexión de nodos y compilación

Luego, conectamos los nodos al punto de inicio (`START`) y final (`END`), y compilamos el grafo en un objeto ejecutable.

### 📊 Visualización y ejecución

Puedes visualizar el grafo generado con `draw_mermaid_png()` y ejecutarlo con `stream`, que irá devolviendo el estado tras cada paso del grafo.

---

## ✅ Código Python completo


In [10]:
# LangGraph básico con un solo nodo de ChatOpenAI (versión compatible)

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_openai import ChatOpenAI
from typing import Annotated, List, TypedDict

# 1. Definir el esquema del estado (forma segura con TypedDict)
class GraphState(TypedDict):
    messages: Annotated[List[BaseMessage], add_messages]

# 2. Instanciar modelo LLM
model = ChatOpenAI(model_name="gpt-4", temperature=0.3)

# 3. Definir el nodo chatbot
def chatbot(state: GraphState):
    answer = model.invoke(state["messages"])
    return {"messages": [answer]}

# 4. Construir el grafo
builder = StateGraph(GraphState)
builder.add_node("chatbot", chatbot)
builder.add_edge(START, "chatbot")
builder.add_edge("chatbot", END)
graph = builder.compile()

# 5. Ejecutar el grafo con mensaje inicial
entrada = {"messages": [HumanMessage(content="¿Qué proveedores ofrecen modelos LLM?")]}
respuesta = graph.invoke(entrada)

# 6. Mostrar respuesta del modelo
for mensaje in respuesta["messages"]:
    print(mensaje.content)


¿Qué proveedores ofrecen modelos LLM?
LLM (Language Model) es un tipo de modelo de aprendizaje automático que se utiliza en el procesamiento del lenguaje natural. Algunos proveedores que ofrecen modelos LLM incluyen:

1. OpenAI: Ofrece GPT-3, uno de los modelos de lenguaje más avanzados disponibles. 

2. Google: Ofrece BERT y T5, que son modelos de lenguaje que pueden ser utilizados para una variedad de tareas de procesamiento de lenguaje natural.

3. Facebook AI: Ofrece RoBERTa, que es una variante de BERT que ha sido optimizada para tener un rendimiento aún mejor.

4. Microsoft: Ofrece Turing, un modelo de lenguaje que ha sido entrenado en una gran cantidad de texto de internet.

5. Hugging Face: Ofrece una variedad de modelos de lenguaje pre-entrenados que pueden ser utilizados para una variedad de tareas de procesamiento de lenguaje natural.

6. IBM: Ofrece Watson, que incluye capacidades de procesamiento de lenguaje natural y puede ser utilizado para una variedad de tareas.

7. Am



### Agregar memoria a `StateGraph`

LangGraph tiene persistencia integrada, que se utiliza de la misma forma tanto para los grafos más simples como para los más complejos. Veamos cómo se aplica esto a la arquitectura inicial.

Vamos a recompilar nuestro grafo, pero ahora adjuntando un *checkpointer*, que es un adaptador de almacenamiento para LangGraph. LangGraph incluye una clase base que cualquier usuario puede extender para crear un adaptador para su base de datos favorita. Hasta el momento de esta escritura, LangGraph incluye varios adaptadores mantenidos por LangChain:

- Un adaptador en memoria, que usaremos en estos ejemplos.
- Un adaptador para SQLite, adecuado para aplicaciones locales y pruebas.
- Un adaptador para Postgres, optimizado para aplicaciones a gran escala.

Muchos desarrolladores también han creado adaptadores para otros sistemas como Redis o MySQL.

Esto nos devuelve un objeto ejecutable con los mismos métodos que el utilizado anteriormente, pero ahora guarda el estado al final de cada paso. Esto significa que cada invocación después de la primera ya no parte desde cero. Cada vez que se llama al grafo, comienza usando el *checkpointer* para recuperar el estado más reciente guardado (si lo hay), y luego combina la nueva entrada con el estado anterior. Solo después de eso, se ejecutan los primeros nodos.

Veamos la diferencia en acción.

Observa el objeto `thread1`, que identifica la conversación actual como perteneciente a un historial específico de interacciones (lo que en LangGraph se conoce como *hilos* o *threads*). Los *threads* se crean automáticamente la primera vez que se usan. Cualquier cadena puede ser un identificador válido para un *thread* (comúnmente se usan UUIDs). La existencia de *threads* te permite lograr algo muy importante en una aplicación con LLM: que múltiples usuarios puedan interactuar con el sistema al mismo tiempo, sin que sus conversaciones se mezclen.

---

In [6]:
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from typing import Annotated, TypedDict
from langchain_core.messages import HumanMessage

# Definimos el estado inicial del grafo
class State(TypedDict):
    messages: Annotated[list, add_messages]

# Inicializamos el grafo con ese estado
builder = StateGraph(State)

# Modelo de lenguaje
model = ChatOpenAI()

# Nodo chatbot
def chatbot(state: State):
    answer = model.invoke(state["messages"])
    return {"messages": [answer]}

# Agregamos el nodo al grafo
builder.add_node("chatbot", chatbot)

# Definimos conexiones (aristas)
builder.add_edge(START, 'chatbot')
builder.add_edge('chatbot', END)

# Compilamos el grafo con almacenamiento en memoria
graph = builder.compile(checkpointer=MemorySaver())

In [7]:
# Creamos una conversación identificada por un thread_id
thread1 = {"configurable": {"thread_id": "1"}}


In [8]:
# Primera invocación
result_1 = graph.invoke(
    {"messages": [HumanMessage("hi, my name is Jack!")]},
    thread1
)
print(result_1)


{'messages': [HumanMessage(content='hi, my name is Jack!', additional_kwargs={}, response_metadata={}, id='b1c62c88-0530-44fe-a8e6-68193b7fa145'), AIMessage(content='Hello Jack! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 14, 'total_tokens': 25, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BNmwEioDZJmctgZOTEuZCJgmmKPYp', 'finish_reason': 'stop', 'logprobs': None}, id='run-437c6f8c-7445-4491-bc04-1b180731733d-0', usage_metadata={'input_tokens': 14, 'output_tokens': 11, 'total_tokens': 25, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}


In [9]:

# Segunda invocación con memoria
result_2 = graph.invoke(
    {"messages": [HumanMessage("what is my name?")]},
    thread1
)
print(result_2)

{'messages': [HumanMessage(content='hi, my name is Jack!', additional_kwargs={}, response_metadata={}, id='b1c62c88-0530-44fe-a8e6-68193b7fa145'), AIMessage(content='Hello Jack! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 14, 'total_tokens': 25, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BNmwEioDZJmctgZOTEuZCJgmmKPYp', 'finish_reason': 'stop', 'logprobs': None}, id='run-437c6f8c-7445-4491-bc04-1b180731733d-0', usage_metadata={'input_tokens': 14, 'output_tokens': 11, 'total_tokens': 25, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}), HumanMessage(content='what is my name?', additional_kwar



## 🧠 ¿Qué significa “agregar memoria” a un grafo?

Cuando tú hablas con un chatbot, esperas que **recuerde lo que le dijiste antes**, ¿cierto?

Por ejemplo:

1. Tú dices: “Hola, me llamo Jack.”
2. Luego preguntas: “¿Cómo me llamo?”

Un chatbot sin memoria **olvidaría** lo que dijiste y no podría responderte bien.

Pero con memoria, el sistema recuerda que tú dijiste “me llamo Jack” y puede responderte:  
👉 “Te llamas Jack.”

---

## 🧰 ¿Cómo se logra eso en LangGraph?

### Paso 1 – Crear el grafo como siempre

Armas tu grafo con tus nodos, reglas y conexiones. Hasta ahí todo normal.

### Paso 2 – Cuando lo “compilas”, le agregas memoria

Aquí es donde le dices:  
> “Por favor, **guarda lo que va pasando**, para que no se le olvide lo anterior.”

Eso se hace usando algo como `MemorySaver`, que es una forma simple de decirle:  
🗒️ “Guarda todo en una memoria temporal (RAM) mientras el programa está corriendo.”

Esto permite que el grafo recuerde paso a paso lo que el usuario ha dicho antes.

---

## 🎯 ¿Y cómo sabe qué conversación es de quién?

Imagina que varias personas están hablando con el mismo chatbot al mismo tiempo.

Para que **no se mezclen las historias**, cada conversación se etiqueta con un ID único, llamado `thread_id`.

Es como tener varias conversaciones en WhatsApp: cada chat tiene su propio historial, aunque sea con la misma persona.

Por ejemplo:

```python
{"configurable": {"thread_id": "1"}}
```

Este `thread_id = "1"` le dice al grafo:  
👉 “Esta conversación es la de Jack.”

Si después entra alguien más (digamos Ana), se usaría otro ID, como `"2"`, y así cada quien tiene su propio “hilo de conversación” separado.

---

## 🔁 Ejemplo contado como historia

### Paso 1: Jack empieza a hablar

El usuario (Jack) manda esto al grafo:

```python
graph.invoke(
    {"messages": [HumanMessage("Hola, me llamo Jack.")]},
    {"configurable": {"thread_id": "1"}}
)
```

El grafo responde algo como:  
👉 “¿Cómo puedo ayudarte, Jack?”

Y **guarda internamente** que “Jack se llama Jack”.

---

### Paso 2: Jack hace otra pregunta

Más tarde, Jack dice:

```python
graph.invoke(
    {"messages": [HumanMessage("¿Cómo me llamo?")]},
    {"configurable": {"thread_id": "1"}}
)
```

Y el chatbot responde:

👉 “Te llamas Jack.”

¿Por qué lo sabe?  
Porque está usando el mismo `thread_id = "1"` que se usó antes, y **LangGraph recuerda lo que se dijo** en ese hilo.

---

## 🧠 ¿Qué logras con esto?

- Tu chatbot puede tener **conversaciones largas y coherentes**, sin tener que repetir todo cada vez.
- Cada persona tiene su propio historial, gracias al `thread_id`.
- Estás construyendo un agente que no solo responde… sino que también **recuerda**.

-

Mis disculpas por la confusión, aquí está la traducción del texto al principio y el código al final, junto con la nota:

---

**Modificación del Historial de Chat**

En muchos casos, los mensajes del historial de chat no están en el mejor estado o formato para generar una respuesta precisa del modelo. Para superar este problema, podemos modificar el historial de chat de tres maneras principales: recortando, filtrando y fusionando los mensajes.

### Recorte de Mensajes

Los **modelos de lenguaje (LLMs)** tienen ventanas de contexto limitadas; en otras palabras, hay un número máximo de tokens que los LLMs pueden recibir como entrada. Por lo tanto, el **mensaje final** enviado al modelo no debe exceder ese límite (particular para cada modo), ya que los modelos rechazarán un mensaje demasiado largo o lo truncarán. Además, la información excesiva en el mensaje puede distraer al modelo y llevar a **alucinaciones**.

Una solución efectiva a este problema es limitar el número de mensajes que se recuperan del historial de chat y se agregan al mensaje. En la práctica, solo necesitamos cargar y almacenar los mensajes más recientes.

Afortunadamente, LangChain proporciona la ayuda incorporada **`trim_messages`** que incorpora varias estrategias para cumplir con estos requisitos. Por ejemplo, el **recortador** permite especificar cuántos tokens queremos conservar o eliminar del historial de chat.

Aquí hay un ejemplo que recupera los últimos **`max_tokens`** de la lista de mensajes configurando el parámetro **`strategy`** a "last":

---

**Nota:**

- El parámetro **`strategy`** controla si se debe comenzar desde el principio o el final de la lista. Usualmente, querrás priorizar los mensajes más recientes y recortar los más antiguos si no caben. Es decir, empezar desde el final de la lista. Para este comportamiento, elige el valor **`last`**. La otra opción disponible es **`first`**, que prioriza los mensajes más antiguos y recorta los más recientes si no caben.
  
- El **`token_counter`** es un modelo LLM o de chat, que se utilizará para contar los tokens utilizando el tokenizador apropiado para ese modelo.

- Podemos agregar el parámetro **`include_system=True`** para asegurarnos de que el recortador mantenga el mensaje del sistema.

- El parámetro **`allow_partial`** determina si se debe recortar el contenido del último mensaje para ajustarse al límite. En nuestro ejemplo, lo configuramos en **`False`**, lo que elimina completamente el mensaje que excedería el límite total.

- El parámetro **`start_on="human"`** asegura que nunca se eliminará un **`AIMessage`** (es decir, una respuesta del modelo) sin eliminar también el mensaje correspondiente de **`HumanMessage`** (la pregunta para esa respuesta).

---


In [11]:
from langchain_core.messages import SystemMessage, trim_messages
from langchain_openai import ChatOpenAI
from langchain_core.messages import (
    AIMessage,
    HumanMessage,
    SystemMessage,
)

# Configuración del recorte de mensajes
trimmer = trim_messages(
    max_tokens=65,               # Límite de tokens
    strategy="last",             # Mantener los mensajes más recientes
    token_counter=ChatOpenAI(model="gpt-4o"),  # Contador de tokens para el modelo
    include_system=True,         # Mantener el mensaje del sistema
    allow_partial=False,         # No permitir cortar el último mensaje
    start_on="human"             # Empezar a cortar desde un mensaje de "human"
)

# Ejemplo de mensajes
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="what's 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

# Aplicar el recorte de mensajes
trimmer.invoke(messages)


[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="what's 2 + 2", additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

**Filtrar Mensajes**

A medida que la lista de mensajes del historial de chat crece, se pueden utilizar una mayor variedad de tipos, subcadenas y modelos. El helper **`filter_messages`** de LangChain facilita el filtrado de los mensajes del historial de chat por tipo, ID o nombre.

---

Aquí hay un ejemplo donde filtramos los mensajes humanos:

```python
from langchain_core.messages import (
    AIMessage,
    HumanMessage,
    SystemMessage,
    filter_messages,
)

messages = [
    SystemMessage("you are a good assistant", id="1"),
    HumanMessage("example input", id="2", name="example_user"),
    AIMessage("example output", id="3", name="example_assistant"),
    HumanMessage("real input", id="4", name="bob"),
    AIMessage("real output", id="5", name="alice"),
]

filter_messages(messages, include_types="human")
```

---

Probemos otro ejemplo donde filtramos para excluir ciertos usuarios y IDs, e incluir tipos de mensajes:

```python
filter_messages(messages, exclude_names=["example_user", "example_assistant"])
```

**Salida esperada:**

```python
[SystemMessage(content='you are a good assistant', id='1'),
 HumanMessage(content='real input', name='bob', id='4'),
 AIMessage(content='real output', name='alice', id='5')]
```

---

```python
filter_messages(
    messages, 
    include_types=[HumanMessage, AIMessage], 
    exclude_ids=["3"]
)
```

**Salida esperada:**

```python
[HumanMessage(content='example input', name='example_user', id='2'),
 HumanMessage(content='real input', name='bob', id='4'),
 AIMessage(content='real output', name='alice', id='5')]
```

---

El helper **`filter_messages`** también puede usarse de forma imperativa o declarativa, lo que facilita su composición con otros componentes en una cadena:

```python
model = ChatOpenAI()

filter_ = filter_messages(exclude_names=["example_user", "example_assistant"])

chain = filter_ | model
```






## 🧩 Fusión de mensajes consecutivos (`merge_message_runs`)

Algunos modelos de lenguaje —como los modelos de Anthropic— **no permiten entradas que contengan múltiples mensajes consecutivos del mismo tipo** (por ejemplo, varios `SystemMessage` seguidos, o varios `AIMessage` juntos). Para resolver esto, LangChain proporciona una utilidad llamada `merge_message_runs`, que **fusiona de manera automática** esos mensajes repetidos, creando una secuencia más limpia y compatible.

---

### 📋 ¿Qué hace exactamente?

`merge_message_runs` toma una lista de mensajes como:

- `SystemMessage`
- `HumanMessage`
- `AIMessage`

Y combina los que están **uno seguido del otro y son del mismo tipo**, en un solo mensaje. El contenido de los mensajes se junta, ya sea como una cadena unificada (con saltos de línea) o como una lista de bloques de contenido (si alguno ya era una lista).

---

### 🔎 Ejemplo:

Supón que tienes este flujo de conversación:

- `SystemMessage("eres un buen asistente.")`
- `SystemMessage("siempre respondes con un chiste.")`
- `HumanMessage` con contenido tipo bloque (una lista con `type: text`)
- Otro `HumanMessage` con texto plano
- Dos `AIMessage` seguidos, cada uno con una broma

Al usar `merge_message_runs`, se fusionan todos los mensajes del mismo tipo que están seguidos, resultando en:

- Un único `SystemMessage` con los dos textos unidos por `\n`
- Un único `HumanMessage` que une el bloque de contenido con el texto plano como lista
- Un único `AIMessage` con las dos respuestas concatenadas con salto de línea

---

### 📎 Resultado esperado

```python
[
  SystemMessage(content="you're a good assistant.\nyou always respond with a joke."),
  HumanMessage(content=[
      {'type': 'text', 'text': "i wonder why it's called langchain"},
      'and who is harrison chasing anyway'
  ]),
  AIMessage(content='Well, I guess they thought "WordRope" and "SentenceString" just didn\'t have the same ring to it!\nWhy, he\'s probably chasing after the last cup of coffee in the office!')
]
```

---

### 🧰 Composición con otros componentes

Esta utilidad puede usarse de forma:

- **Imperativa** (simplemente llamándola sobre una lista de mensajes).
- **Declarativa**, como parte de un `chain`, es decir, una cadena de pasos en LangChain.

Esto te permite integrarla con modelos de lenguaje de forma fluida, por ejemplo:

```python
model = ChatOpenAI()
merger = merge_message_runs()
chain = merger | model
```

---

## 🐍 Código en Python (completo):

```python
from langchain_core.messages import (
    AIMessage,
    HumanMessage,
    SystemMessage,
    merge_message_runs,
)

messages = [
    SystemMessage("you're a good assistant."),
    SystemMessage("you always respond with a joke."),
    HumanMessage(
        [{"type": "text", "text": "i wonder why it's called langchain"}]
    ),
    HumanMessage("and who is harrison chasing anyway"),
    AIMessage(
        '''Well, I guess they thought "WordRope" and "SentenceString" just 
        didn't have the same ring to it!'''
    ),
    AIMessage("""Why, he's probably chasing after the last cup of coffee in the 
        office!"""),
]

merged = merge_message_runs(messages)
```
